In [1]:
%load_ext autoreload
%autoreload 2

In [89]:
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [90]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import csim as cs

In [114]:
from functools import partial

In [94]:
import qutip

In [112]:
u = cs.random_unitary_gate(2)
m = cs.transfer_matrix(u)

In [169]:
m_choi = cs.matrix_to_choi(m)

In [154]:
for qo in qutip.choi_to_kraus(qutip.Qobj(m)):
    display(qo)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[-0.29925776+0.29986139j  0.4660761 +0.50450932j]
 [ 0.20165963+0.19567308j  0.30184178-0.23065292j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[-0.29925776-0.29986139j  0.20165963-0.19567308j]
 [ 0.4660761 -0.50450932j  0.30184178+0.23065292j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 7.07106781e-01-3.62237036e-16j  1.07247595e-16+2.76767987e-17j]
 [-4.56667179e-16+1.21518444e-16j  7.07106781e-01+2.09136465e-17j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.52527398-2.17852594e-17j -0.02585814+5.20717096e-01j]
 [-0.02585814-5.20717096e-01j -0.12146498+7.60768947e-17j]]

In [171]:
sum([m.data.T.conjugate() @ m.data for m in qutip.choi_to_kraus(qutip.Qobj(m))]).toarray()

array([[1.95738295+0.j        , 0.03974224+0.19276843j],
       [0.03974224-0.19276843j, 1.62589828+0.j        ]])

In [172]:
tot_qo = qutip.choi_to_kraus(qutip.Qobj(m_choi))

for qo in tot_qo:
    display(qo)

# only two relevant

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.69187231+2.61554510e-18j -0.52887306-1.82272792e-01j]
 [ 0.54946132+2.62097781e-01j  0.34387395+5.63348306e-01j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.00118316+6.37645541e-03j  0.03435207+4.97126111e-02j]
 [ 0.06130892+4.76894799e-18j -0.04672341+1.56042605e-02j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 4.45503101e-10-1.30067310e-08j  1.84234504e-09-2.80161509e-09j]
 [-2.19234505e-09+4.63810508e-09j -5.13447622e-09+3.39055168e-09j]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 1.10845686e-10+7.34264300e-10j  2.46288708e-09+9.14480209e-10j]
 [-3.82116601e-11+2.07265974e-09j  2.78485351e-09-1.55026369e-10j]]

In [167]:
# Correct value
sum([m.data.T.conjugate() @ m.data for m in tot_qo]).toarray()

array([[1.00000000e+00+0.00000000e+00j, 1.82040312e-16-3.10304303e-16j],
       [1.82040312e-16+3.10304303e-16j, 1.00000000e+00+0.00000000e+00j]])

In [180]:
kraus_mats = [m.data.toarray() for m in tot_qo[:2]]

In [188]:
for i, km in enumerate(kraus_mats):
    evals = np.linalg.eigvals(km)
    print(i,evals)

0 [0.79517054-0.32559882j 0.24057572+0.88894712j]
1 [ 0.03440991+0.03570869j -0.07995016-0.01372798j]
